# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dikhil,11.5000,42.1667,22.70,57,13,2.20,DJ,1730693632
1,1,bethel,41.3712,-73.4140,3.67,68,40,1.43,US,1730693253
2,2,nar'yan-mar,67.6713,53.0870,-2.11,88,100,7.31,RU,1730693634
3,3,new norfolk,-42.7826,147.0587,19.28,40,35,4.78,AU,1730693278
4,4,udachny,66.4167,112.4000,-20.27,96,80,2.99,RU,1730693424


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
# %%capture --no-display

# Configure the map with cities as the legend
city_map = city_data_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",  # Size of each point represents the humidity in each city
    scale=1,
    color="City",  # Use the 'City' column to group points and create a legend by city
    hover_cols=["City", "Country", "Humidity"]  # Add 'City', 'Country', and 'Humidity' to hover information
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit the criteria and drop any results with null values
# Ideal weather conditions:
# - Max temperature between 21 and 27 degrees
# - Wind speed less than 4.5 m/s
# - Cloudiness at 0%

ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= 21) & 
    (city_data_df["Max Temp"] <= 27) & 
    (city_data_df["Wind Speed"] < 4.5) & 
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head(20)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,ingeniero guillermo n. juarez,-23.9000,-61.8500,23.67,76,0,1.72,AR,1730693641
46,46,morondava,-20.2833,44.2833,26.35,70,0,1.31,MG,1730693689
143,143,as sulayyil,20.4607,45.5779,22.38,34,0,1.79,SA,1730693817
302,302,remire-montjoly,4.9167,-52.2667,26.02,89,0,1.54,GF,1730693553
307,307,kandahar,31.0000,65.7500,23.02,20,0,0.47,AF,1730694035
354,354,ixtapa,20.7000,-105.2000,25.57,69,0,0.00,MX,1730694098
382,382,wesley chapel,28.2397,-82.3279,22.72,89,0,2.57,US,1730694109
383,383,barela,23.1000,80.0500,22.45,68,0,1.03,IN,1730694140
453,453,taoudenni,22.6783,-3.9836,24.74,17,0,1.02,ML,1730694228
484,484,san martin,-33.0810,-68.4681,21.03,62,0,3.18,AR,1730694269


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(20)


,City,Country,Lat,Lng,Humidity,Hotel Name
8,ingeniero guillermo n. juarez,AR,-23.9000,-61.8500,76,
46,morondava,MG,-20.2833,44.2833,70,
143,as sulayyil,SA,20.4607,45.5779,34,
302,remire-montjoly,GF,4.9167,-52.2667,89,
307,kandahar,AF,31.0000,65.7500,20,
354,ixtapa,MX,20.7000,-105.2000,69,
382,wesley chapel,US,28.2397,-82.3279,89,
383,barela,IN,23.1000,80.0500,68,
453,taoudenni,ML,22.6783,-3.9836,17,
484,san martin,AR,-33.0810,-68.4681,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000       # Radius in meters
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ingeniero guillermo n. juarez - nearest hotel: Hotel El Indiano
morondava - nearest hotel: Hotel Sharon Menabe
as sulayyil - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
kandahar - nearest hotel: No hotel found
ixtapa - nearest hotel: Ma. Cristina
wesley chapel - nearest hotel: Sleep Inn
barela - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
san martin - nearest hotel: Hotel Lacava
brandon - nearest hotel: Brandon Motor Lodge
tidjikja - nearest hotel: No hotel found
antonio enes - nearest hotel: Hotel Quirimbas
son la - nearest hotel: Khách sạn Mường Thanh Luxury Sơn La


,City,Country,Lat,Lng,Humidity,Hotel Name
8,ingeniero guillermo n. juarez,AR,-23.9000,-61.8500,76,Hotel El Indiano
46,morondava,MG,-20.2833,44.2833,70,Hotel Sharon Menabe
143,as sulayyil,SA,20.4607,45.5779,34,No hotel found
302,remire-montjoly,GF,4.9167,-52.2667,89,Complexe Belova
307,kandahar,AF,31.0000,65.7500,20,No hotel found
354,ixtapa,MX,20.7000,-105.2000,69,Ma. Cristina
382,wesley chapel,US,28.2397,-82.3279,89,Sleep Inn
383,barela,IN,23.1000,80.0500,68,No hotel found
453,taoudenni,ML,22.6783,-3.9836,17,No hotel found
484,san martin,AR,-33.0810,-68.4681,62,Hotel Lacava


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
# %%capture --no-display

# Configure the map plot
city_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",    # Size of the points based on humidity
    color="City",       # Color based on each city
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    hover_cols=["City", "Country", "Hotel Name", "Humidity"]
)

# Display the map
city_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)